# Import Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data

In [2]:
df = pd.read_csv("NYSE_train.csv")
#df1 = pd.read_csv("NASDAQ_test.csv")
SPY = pd.read_csv("SPY.csv")
Rf = pd.read_csv("Risk_Free.csv")

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
df.head(3)

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,Net Operating Cash Flow Growth
0,10051,1991-01-01,HNGR,4.875,-0.025000,NaN,NaN,NaN,1.672589,0.241117,1.047300,0.069424,0.007759,0.988372,-0.83299,-0.284173
1,10051,1991-02-01,HNGR,4.500,-0.076923,NaN,NaN,NaN,1.672589,0.241117,1.047300,0.069424,0.007759,0.988372,-0.83299,-0.284173
2,10051,1991-03-01,HNGR,6.250,0.388889,2.282,NaN,NaN,2.138354,0.226499,1.355927,0.058706,0.215127,-0.583626,8.00000,-3.442211


In [4]:
len(df)

709274

In [5]:
df1 = df.copy()[['PERMNO', 'Date', 'TICKER', 'Return']]  # Save time

In [6]:
df1.head(3)

,PERMNO,Date,TICKER,Return
0,10051,1991-01-01,HNGR,-0.025000
1,10051,1991-02-01,HNGR,-0.076923
2,10051,1991-03-01,HNGR,0.388889


In [7]:
SPY.head(3)

,Date,SPY Return
0,1990-02-01,0.0085
1,1990-03-01,0.0243
2,1990-04-01,-0.0269


In [8]:
Rf.head(3)

,Date,Rf
0,1990-01-01,0.006958
1,1990-02-01,0.007033
2,1990-03-01,0.007208


# Rolling Window and Calculate Var & Cov

In [9]:
df1 = pd.merge(df1, SPY, on = 'Date', how = 'outer')
df1 = pd.merge(df1, Rf, on = 'Date', how = 'outer')

In [10]:
df1.dropna(subset=['TICKER'], inplace=True)

In [11]:
len(df1)

704424

In [12]:
df1.head(3)

,PERMNO,Date,TICKER,Return,SPY Return,Rf
0,10051.0,1991-01-01,HNGR,-0.025000,0.0415,0.00635
1,10057.0,1991-01-01,AMT.1,0.152632,0.0415,0.00635
2,10064.0,1991-01-01,TTI.4,0.190476,0.0415,0.00635


In [13]:
df1.reset_index(inplace=True, drop=True)
df1.head(3)

,PERMNO,Date,TICKER,Return,SPY Return,Rf
0,10051.0,1991-01-01,HNGR,-0.025000,0.0415,0.00635
1,10057.0,1991-01-01,AMT.1,0.152632,0.0415,0.00635
2,10064.0,1991-01-01,TTI.4,0.190476,0.0415,0.00635


In [ ]:
a = df1.groupby(by='TICKER')['Return'].rolling(window=36).cov(df1['SPY Return'])

In [14]:
def rolling(df_):
    df = df_.copy()
    TICKER = list(set(df['TICKER']))
    dataframe = df[df['TICKER'] == TICKER[0]].copy()
    dataframe['Var'] = dataframe['SPY Return'].rolling(window=36).var()
    dataframe['Cov'] = dataframe['Return'].rolling(window=36).cov(dataframe['SPY Return'])
    dataframe['Beta'] = dataframe['Cov'] / dataframe['Var']
    dataframe['Alpha'] = dataframe['Return'] - dataframe['Rf'] - dataframe['Beta'] * (dataframe['SPY Return'] - dataframe['Rf'])
    
    for i in range(1, len(TICKER)):
        #DF = df.loc[TICKER[i]]
        DF = df[df['TICKER'] == TICKER[i]].copy()
        if len(DF) < 36:
            continue
        DF['Var'] = DF['SPY Return'].rolling(window=36).var()
        DF['Cov'] = DF['Return'].rolling(window=36).cov(DF['SPY Return'])
        DF['Beta'] = DF['Cov'] / DF['Var']
        DF['Alpha'] = DF['Return'] - DF['Rf'] - DF['Beta'] * (DF['SPY Return'] - DF['Rf']) 
        
        dataframe = pd.concat([dataframe, DF])
        
    return dataframe

In [15]:
df2 = rolling(df1)

In [16]:
df2.reset_index(inplace=True, drop=True)
df2

,PERMNO,Date,TICKER,Return,SPY Return,Rf,Var,Cov,Beta,Alpha
0,86176.0,1998-06-01,KG,0.000000,0.0394,0.004545,NaN,NaN,NaN,NaN
1,86176.0,1998-07-01,KG,0.151786,-0.0116,0.004583,NaN,NaN,NaN,NaN
2,86176.0,1998-08-01,KG,-0.108527,-0.1458,0.004076,NaN,NaN,NaN,NaN
3,86176.0,1998-09-01,KG,-0.043478,0.0624,0.003520,NaN,NaN,NaN,NaN
4,86176.0,1998-10-01,KG,0.131818,0.0803,0.003524,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
689290,75814.0,2000-11-01,PDI.,-0.033801,-0.0801,0.004494,0.002594,-7.921460e-08,-0.000031,-0.038298
689291,75814.0,2000-12-01,PDI.,0.109628,0.0041,0.004151,0.002594,-1.490513e-05,-0.005745,0.105477
689292,75814.0,2001-01-01,PDI.,-0.052400,0.0346,0.004032,0.002611,-5.444369e-05,-0.020849,-0.055794
689293,75814.0,2001-02-01,PDI.,0.040977,-0.0923,0.003875,0.002790,-1.968974e-04,-0.070569,0.030315


In [17]:
df3 = df2.copy()[['PERMNO', 'Date', 'Beta', 'Alpha']]

In [18]:
df3.head(3)

,PERMNO,Date,Beta,Alpha
0,86176.0,1998-06-01,NaN,NaN
1,86176.0,1998-07-01,NaN,NaN
2,86176.0,1998-08-01,NaN,NaN


In [26]:
df4 = pd.merge(df, df3, on = ['PERMNO', 'Date'], how = 'outer')

In [27]:
df4

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,Net Operating Cash Flow Growth,Beta,Alpha
0,10051,1991-01-01,HNGR,4.875,-0.025000,NaN,NaN,NaN,1.672589,0.241117,1.047300,0.069424,0.007759,0.988372,-0.832990,-0.284173,NaN,NaN
1,10051,1991-02-01,HNGR,4.500,-0.076923,NaN,NaN,NaN,1.672589,0.241117,1.047300,0.069424,0.007759,0.988372,-0.832990,-0.284173,NaN,NaN
2,10051,1991-03-01,HNGR,6.250,0.388889,2.282,NaN,NaN,2.138354,0.226499,1.355927,0.058706,0.215127,-0.583626,8.000000,-3.442211,NaN,NaN
3,10051,1991-04-01,HNGR,4.500,-0.280000,2.282,8413.672217,5.258545e+04,2.138354,0.226499,1.355927,0.058706,0.215127,-0.583626,8.000000,-3.442211,NaN,NaN
4,10051,1991-05-01,HNGR,4.875,0.083333,2.282,24539.877300,1.104294e+05,2.138354,0.226499,1.355927,0.058706,0.215127,-0.583626,8.000000,-3.442211,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709317,93427,2015-08-01,FN,19.880,0.071121,35.438,82075.145320,1.523315e+06,2.911046,1.413128,2.189618,0.005840,0.001047,-0.056974,0.242837,0.322865,0.976723,0.132235
709318,93427,2015-09-01,FN,18.330,-0.077968,35.777,127916.558500,2.542981e+06,2.778029,1.282151,2.035520,0.089748,0.084289,0.079232,0.201936,0.300091,1.093651,-0.048988
709319,93427,2015-10-01,FN,21.670,0.182215,35.777,116566.229700,2.136659e+06,2.778029,1.282151,2.035520,0.089748,0.084289,0.079232,0.201936,0.300091,1.126153,0.088905
709320,93427,2015-11-01,FN,23.930,0.104292,35.777,181662.408800,3.936624e+06,2.778029,1.282151,2.035520,0.089748,0.084289,0.079232,0.201936,0.300091,1.167730,0.103939


# Drop Missing Values

In [51]:
df5 = df4.dropna(how='any')

In [52]:
len(df5)

305256

In [53]:
df5 

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,Net Operating Cash Flow Growth,Beta,Alpha
35,10051,1993-12-01,HNGR,6.250,-0.074074,8.258,59433.27682,4.011746e+05,4.155172,0.281476,2.501002,0.003731,0.020530,-0.083658,-0.262206,0.276680,1.335219,-0.086106
36,10051,1994-01-01,HNGR,6.250,0.000000,8.258,64882.53814,4.055159e+05,4.155172,0.281476,2.501002,0.003731,0.020530,-0.083658,-0.262206,0.276680,1.384367,-0.043386
37,10051,1994-02-01,HNGR,6.250,0.000000,8.258,68321.62751,4.270102e+05,4.155172,0.281476,2.501002,0.003731,0.020530,-0.083658,-0.262206,0.276680,1.683698,0.053698
38,10051,1994-03-01,HNGR,6.000,-0.040000,8.291,41511.26181,2.594454e+05,3.676069,0.069186,1.996097,0.071707,0.273051,-0.375796,-1.056373,0.702786,1.463901,0.029314
39,10051,1994-04-01,HNGR,5.625,-0.062500,8.291,94729.22446,5.683753e+05,3.676069,0.069186,1.996097,0.071707,0.273051,-0.375796,-1.056373,0.702786,1.404645,-0.076416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709317,93427,2015-08-01,FN,19.880,0.071121,35.438,82075.14532,1.523315e+06,2.911046,1.413128,2.189618,0.005840,0.001047,-0.056974,0.242837,0.322865,0.976723,0.132235
709318,93427,2015-09-01,FN,18.330,-0.077968,35.777,127916.55850,2.542981e+06,2.778029,1.282151,2.035520,0.089748,0.084289,0.079232,0.201936,0.300091,1.093651,-0.048988
709319,93427,2015-10-01,FN,21.670,0.182215,35.777,116566.22970,2.136659e+06,2.778029,1.282151,2.035520,0.089748,0.084289,0.079232,0.201936,0.300091,1.126153,0.088905
709320,93427,2015-11-01,FN,23.930,0.104292,35.777,181662.40880,3.936624e+06,2.778029,1.282151,2.035520,0.089748,0.084289,0.079232,0.201936,0.300091,1.167730,0.103939


In [54]:
df5 = pd.merge(df5, SPY, on = 'Date', how = 'outer')
df5 = pd.merge(df5, Rf, on = 'Date', how = 'outer')

df5.dropna(how='any', inplace=True)

In [55]:
df5.to_csv("df_train.csv", index=False)